# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [ ]:
!python -m pip install semantic-kernel -U

In [23]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion("gpt-35-turbo", endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

## It all begins with an ask

In [24]:
ask = """
Convert the text to uppercase
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in Vietnamese.
"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [25]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [26]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [27]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [28]:
print(basic_plan.generated_plan)

{
    "input": "Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in Vietnamese.",
    "subtasks": [
        {"function": "TextSkill.uppercase"},
        {"function": "WriterSkill.NovelOutline"},
        {"function": "WriterSkill.Translate", "args": {"language": "Vietnamese"}}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [29]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [30]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks Vietnamese so write it in Vietnamese.
Convert the text to uppercase.
"""

new_plan = await planner.create_plan_async(ask, kernel)

In [31]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "Vietnamese"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [32]:
results = await planner.execute_plan_async(new_plan, kernel)

In [33]:
print(results)

1. CẦU NGUYỆN, THAM GIA MỘT BỮA TIỆC LÃNG MẠN TẠI MỘT NHÀ HÀNG QUÝ TỘC
2. MỘT CUỘC HẸN GIỮA VẺ ĐẸP TỰ NHIÊN, MỘT BUỔI PICNIC TRONG MỘT CÔNG VIÊN CÔNG BẰNG
3. MỘT NGÀY ĐƯỢC CHĂM SÓC CHO CẶP ĐÔI ĐÍNH HÔN NÀY, MỘT NGÀY TẠI SPA THẦN THÁNH
4. CHÚNG TA SẼ HÀNH TRÌNH ĐẾN MỘT VƯỜN NHO ĐỊA PHƯƠNG, ĐỂ THƯỞNG THỨC MẬT ONG CỦA CÁC VỊ THẦN
5. CÙNG NHAU, HÃY CHUẨN BỊ MỘT BỮA TIỆC TỰ NẤU, VỚI HAI TAY VÀ HAI TRÁI TIM CỦA CHÚNG TA LIÊN KẾT VỚI NHAU
6. MỘT CUỘC ĐI DẠO GIỮA NHỮNG NGỌN ĐỒI XANH TƯƠI, MỘT CUỘC LEO NÚI HOẶC MỘT CUỘC DẠO CHƠI TRONG THIÊN NHIÊN
7. TRONG SỰ THOẢI MÁI CỦA NGÔI NHÀ, HÃY THƯỞNG THỨC MỘT CUỘC MARATHON CÁC BỘ PHIM TÌNH YÊU
8. MỘT ĐIỆU NHẢY, MỘT VŨ ĐIỆU QUÝ TỘC, CHÚNG TA SẼ HỌC NHẢY NHƯ MỘT
9. LÊN THUYỀN, KHI MẶT TRỜI LẶN, TÌNH YÊU CỦA CHÚNG TA TRÔI TRÊN NHỮNG DÒNG NƯỚC YÊN BÌNH
10. TRONG MỘT PHÒNG TRƯNG BÀY HOẶC BẢO TÀNG NGHỆ THUẬT, HÃY NGÂM MÌNH TRONG VÒNG TAY VĂN HÓA


# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [34]:
from semantic_kernel.planning import SequentialPlanner

planner = SequentialPlanner(kernel)

In [35]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

In [36]:
sequential_plan = await planner.create_plan_async(goal=ask)

PlanningException: (<ErrorCodes.CreatePlanError: 3>, ('Not possible to create plan for goal with available functions.\n', "Goal:\nTomorrow is Valentine's day. I need to come up with a few date ideas.\nShe likes Shakespeare so write using his style. She speaks French so write it in French.\nConvert the text to uppercase.\nFunctions:\nSummarizeSkill.MakeAbstractReadable:\n  description: Given a scientific white paper abstract, rewrite it to make it more readable\n  inputs:\n    - input: \n\nSummarizeSkill.Summarize:\n  description: Summarize given text or any text document\n  inputs:\n    - input: Text to summarize\n\nSummarizeSkill.Topics:\n  description: Analyze given text or document and extract key topics worth remembering\n  inputs:\n    - input: \n\nSummarizeSkill.Notegen:\n  description: Automatically generate compact notes for any text or text document.\n  inputs:\n    - input: \n\nWriterSkill.NovelChapter:\n  description: Write a chapter of a novel.\n  inputs:\n    - input: A sy

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [ ]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Let's ask the sequential planner to execute the plan.

In [ ]:
result = await sequential_plan.invoke_async()

In [ ]:
print(result)

# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [37]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [38]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10fb6bed0>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10dd60a10>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10de35c50>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10de2fcd0>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10fadae10>}

In [39]:
ask = "What is the sum of 110 and 990?"

In [40]:
plan = await planner.create_plan_async(goal=ask)

In [41]:
result = await plan.invoke_async()

In [42]:
print(result)

1100
